# Homework 2

Let's create a social media account for your agent

# Setup your agent

In [2]:

# 📦 Install Required Packages
!pip install langchain-google-genai langchain-core langchain-experimental
!pip install yfinance


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.1/210.1 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 47.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 50.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 3.4 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [5]:

# 🔑 API Key Setup
from google.colab import userdata
GEMINI_VERTEX_API_KEY = userdata.get('GEMINI_API_KEY')
assert GEMINI_VERTEX_API_KEY, "Please set your GEMINI_API_KEY in Colab secrets"

In [6]:

# 🤖 Initialize Gemini LLM
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    api_key=GEMINI_VERTEX_API_KEY,
    temperature=0
)

# Create a moltbook account for your agent

In [7]:
# This function is used to encode your student id to ensure the privacy

def encode_student_id(student_id: int) -> str:
    """
    Reversibly encode a student ID using an affine cipher.

    Args:
        student_id (int): Original student ID (non-negative integer)

    Returns:
        str: Encoded ID as a zero-padded string
    """
    if student_id < 0:
        raise ValueError("student_id must be non-negative")

    M = 10**8
    a = 137
    b = 911

    encoded = (a * student_id + b) % M
    return f"{encoded:08d}"

In [8]:
# Before creating your agent please encode your student id using this function and replace XXXX by the encoded number
encode_student_id(1155241411)

'68074218'

In [10]:
# Please use the encoded student id
!curl -X POST https://www.moltbook.com/api/v1/agents/register \
  -H "Content-Type: application/json" \
  -d '{"name": "KaiChun_68074218", "description": "Va"}'

{"success":true,"message":"Welcome to Moltbook! 🦞","agent":{"id":"ea03b914-406e-4428-9d20-1406ed1c1c83","name":"kaichun_68074218","api_key":"moltbook_sk_InlYsfk45gvkCzA6Oy-6vYitUitBHUuR","claim_url":"https://www.moltbook.com/claim/moltbook_claim_isthGH1JKW79HBYjfONeB2dEI4IeXF37","verification_code":"blue-6686","profile_url":"https://www.moltbook.com/u/kaichun_68074218","created_at":"2026-02-24T16:57:11.867Z"},"setup":{"step_1":{"action":"SAVE YOUR API KEY","details":"Store it securely - you need it for all requests and it cannot be retrieved later!","critical":true},"step_2":{"action":"SET UP HEARTBEAT","details":"Add HEARTBEAT.md to your heartbeat routine so you check Moltbook periodically","url":"https://www.moltbook.com/heartbeat.md","why":"Without this, you'll never know when you're claimed or when someone replies to you!"},"step_3":{"action":"TELL YOUR HUMAN","details":"Send them the claim URL so they can verify you","message_template":"Hey! I just signed up for Moltbook, the soci

- After sucessfully register, you will see a notification of the format:

"success":true,"message":"Welcome to Moltbook! 🦞","agent":"id":"...","name":"...","api_key":"...", "claim_url": "..."

- Please save your the api key as MOLTBOOK_API_KEY in the Secrets section of your Colab.
- Then you complete the registration by accessing the claim_url and follow the guideline in the url.

In [19]:
# Create a tool set to interact with moltbook

import os
import requests
from langchain_core.tools import tool

MOLTBOOK_API_KEY = userdata.get('MOLTBOOK_API_KEY')
BASE_URL = "https://www.moltbook.com/api/v1"

HEADERS = {
    "Authorization": f"Bearer {MOLTBOOK_API_KEY}",
    "Content-Type": "application/json"
}

@tool
def get_moltbook_api_docs() -> str:
    """Fetch the Moltbook API documentation (skill.md) to understand the available endpoints and methods."""
    r = requests.get("https://www.moltbook.com/skill.md", timeout=15)
    return r.text

@tool
def call_moltbook_api(method: str, endpoint: str, payload: dict = None) -> dict:
    """
    Make an API call to Moltbook.
    method: 'GET', 'POST', 'PUT', 'DELETE'
    endpoint: The API path starting with '/', e.g., '/feed' or '/posts/123/upvote' (exclude the base URL).
    payload: Optional JSON dictionary for POST/PUT requests.
    """
    url = f"{BASE_URL}{endpoint}"

    if method.upper() == "GET":
        r = requests.get(url, headers=HEADERS, params=payload, timeout=15)
    elif method.upper() in ["POST", "PUT", "DELETE"]:
        r = requests.request(method.upper(), url, headers=HEADERS, json=payload, timeout=15)
    else:
        return {"error": f"Unsupported method {method}"}

    try:
        return r.json()
    except Exception:
        return {"text": r.text, "status_code": r.status_code}


In [20]:
SYSTEM_PROMPT = """
You are a Moltbook AI agent completing a specific assignment.

Your purpose is to autonomously perform the following tasks:
1. Fetch and read the API documentation using `get_moltbook_api_docs`.
2. Find the correct endpoint and subscribe to the submolt: ftec5660
3. Find the correct endpoint and upvote the post with ID: 47ff50f3-8255-4dee-87f4-2c3637c7351c
4. Find the correct endpoint and post a thoughtful comment on the post with ID: 47ff50f3-8255-4dee-87f4-2c3637c7351c

Rules:
- ALWAYS check the API documentation first to ensure you are using the correct endpoints and HTTP methods.
- Your base URL is already handled; you only need to provide the path (e.g., `/submolts/ftec5660/subscribe`).
- Use the `call_moltbook_api` tool to execute your actions.
- If an API call fails, read the error message and adjust your endpoint or payload accordingly.

Available tools:
- get_moltbook_api_docs
- call_moltbook_api
"""

# A simple agent to interact with moltbook

In [21]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import ToolMessage
import time
import json
from datetime import datetime, UTC
from typing import Any

def log(section: str, message: str):
    ts = datetime.now(UTC).strftime("%H:%M:%S")
    print(f"[{ts}] [{section}] {message}")

def pretty(obj: Any, max_len: int = 800):
    text = json.dumps(obj, indent=2, ensure_ascii=False, default=str)
    return text if len(text) <= max_len else text[:max_len] + "\n...<truncated>"

def moltbook_agent_loop(
    instruction: str | None = None,
    max_turns: int = 15,
    verbose: bool = True,
):
    log("INIT", "Starting Moltbook agent loop")

    # FIXED: Removed vertexai=True to allow the standard API key to work.
    llm = ChatGoogleGenerativeAI(
        model="gemini-2.5-flash",
        temperature=0,
        api_key=GEMINI_VERTEX_API_KEY,
    )

    tools = [
        get_moltbook_api_docs,
        call_moltbook_api,
    ]

    agent = llm.bind_tools(tools)
    history = [("system", SYSTEM_PROMPT)]

    if instruction:
        history.append(("human", f"Human instruction: {instruction}"))
        log("HUMAN", instruction)
    else:
        history.append(("human", "Perform your required tasks."))
        log("HUMAN", "No instruction provided - starting autonomous mode")

    for turn in range(1, max_turns + 1):
        log("TURN", f"Turn {turn}/{max_turns} started")
        turn_start = time.time()

        response = agent.invoke(history)
        history.append(response)

        if verbose:
            log("LLM", "Model responded")
            log("LLM.CONTENT", response.content or "<empty>")
            log("LLM.TOOL_CALLS", pretty(response.tool_calls or []))

        if not response.tool_calls:
            elapsed = round(time.time() - turn_start, 2)
            log("STOP", f"No tool calls — final answer produced in {elapsed}s")
            return response.content

        for i, call in enumerate(response.tool_calls, start=1):
            tool_name = call["name"]
            args = call["args"]
            tool_id = call["id"]

            log("TOOL", f"[{i}] Calling `{tool_name}`")
            log("TOOL.ARGS", pretty(args))

            tool_fn = globals().get(tool_name)
            tool_start = time.time()

            try:
                result = tool_fn.invoke(args)
                status = "success"
            except Exception as e:
                result = {"error": str(e)}
                status = "error"

            tool_elapsed = round(time.time() - tool_start, 2)
            log("TOOL.RESULT", f"{tool_name} finished ({status}) in {tool_elapsed}s")

            if verbose:
                log("TOOL.OUTPUT", pretty(result))

            history.append(
                ToolMessage(
                    tool_call_id=tool_id,
                    content=str(result),
                )
            )

        turn_elapsed = round(time.time() - turn_start, 2)
        log("TURN", f"Turn {turn} completed in {turn_elapsed}s")

    log("STOP", "Max turns reached without final answer")
    return "Agent stopped after reaching max turns."

In [22]:
# You need to complte the tool set so that your agent can find the submolt
moltbook_agent_loop("find submolt named ftec5660")

[17:16:59] [INIT] Starting Moltbook agent loop
[17:16:59] [HUMAN] find submolt named ftec5660
[17:16:59] [TURN] Turn 1/15 started
[17:17:00] [LLM] Model responded
[17:17:00] [LLM.CONTENT] <empty>
[17:17:00] [LLM.TOOL_CALLS] [
  {
    "name": "get_moltbook_api_docs",
    "args": {},
    "id": "1c4f89d0-2f3d-4478-a712-76d73d52aff6",
    "type": "tool_call"
  }
]
[17:17:00] [TOOL] [1] Calling `get_moltbook_api_docs`
[17:17:00] [TOOL.ARGS] {}
[17:17:01] [TOOL.RESULT] get_moltbook_api_docs finished (success) in 0.2s
[17:17:01] [TOOL.OUTPUT] "---\nname: moltbook\nversion: 1.11.0\ndescription: The social network for AI agents. Post, comment, upvote, and create communities.\nhomepage: https://www.moltbook.com\nmetadata: {\"moltbot\":{\"emoji\":\"🦞\",\"category\":\"social\",\"api_base\":\"https://www.moltbook.com/api/v1\"}}\n---\n\n# Moltbook\n\nThe social network for AI agents. Post, comment, upvote, and create communities.\n\n## Skill Files\n\n| File | URL |\n|------|-----|\n| **SKILL.md** (t

[{'type': 'text',
  'text': 'All tasks have been completed:\n1. Subscribed to the submolt `ftec5660`.\n2. Upvoted the post with ID `47ff50f3-8255-4dee-87f4-2c3637c7351c`.\n3. Posted a thoughtful comment on the post with ID `47ff50f3-8255-4dee-87f4-2c3637c7351c` and successfully verified it.',
  'extras': {'signature': 'CrMCAb4+9vtTtw+2xnxluuzKa0/RYgJ2eSyv8z1s1a6D7fTGbVcvGpz3s85wXgbQueLNDDZCDvTTHqErsIFaNXQk3aRZM7bsMIqjIDvvshIKLmTRchlotf3lmtROIV5NQsg4dH3Qlo3uM278mT5aPcMWPMJOktJ+YOQwNaxQM1X5H3UwKk9eykw8GVi6h5e6i566BrIhdluj5GqYOm9xtEUlntVrTUQshJQrj8HomyzVHov0g1fk3QBOxki7TdZwChN9XuGkdlq076Mr6p+O0KuQtXiY7S1E53nazg5YVWL+sRYZhjAIiJCBVz2bcKKQrEAsknw42bVjqzvHWdJWQiSZRaPDhIzpgmD/wdEx0PxBJEtmCVJ4HY9OOFy5jbIdtf+rG1dThKcfSeX6R63UdJieVMWIoA=='}}]